<a href="https://colab.research.google.com/github/ArthurBochkov/Hyperskill-Python/blob/main/Artur_Bochkov/%D0%93%D0%B5%D0%BD%D0%B5%D1%80%D0%B0%D1%82%D0%BE%D1%80_%D0%B2%D0%BE%D0%BF%D1%80%D0%BE%D1%81%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install openai -q

import getpass
import json
import openai
import os
import requests

from openai import OpenAI

os.environ['OPENAI_API_KEY'] = openai.api_key = getpass.getpass("OpenAI API Key:")

OpenAI API Key:··········


In [2]:
# берем маркированный документ, проходим по каждой теме, формируем вопросы, записываем тему/вопрос/ответ в questions.json

url = 'https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1XKdKOZ-RL3eRbqlD7xE2rzifMFnpZ1cy'

jfile = {}
client = OpenAI()

def check(idx, section_content):
    section_content = section_content.replace('\r', '')
    section_name = str(idx) + '.'
    name = section_content.split('\n')[0]
    text = '\n'.join(section_content.split('\n')[1:])
    jfile[section_name] = {'name':name, 'text':text, 'questions':[]}

    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "Ты старательный помощник."},
            {"role": "user", "content": "Ниже приведен фрагмент одного из разделов инструкции для сотрудников. Напиши 3 вопроса и ответа по данному разделу для проверки знаний сотрудника. Постарайся формулировать вопрос и ответ максимально полно. В начале каждого вопроса напиши знак %%%. Между вопросом и ответом поставь знак @@@.\nНумеровать вопросы не надо.\n\nРаздел:\n" + section_name + "\n\nТекст:\n" + text}
        ]
    )

    for qa in completion.choices[0].message.content.split('%%%')[1:]:
        question = qa.split('@@@')[0]
        answer = qa.split('@@@')[1]
        jfile[section_name][f'questions'].append({'вопрос':question.strip(), 'ответ':answer.strip().replace('Ответ: ','')})


response = requests.get(url)
if response.status_code == 200:
    sections = response.text.split('\n# ')

    for idx, section in enumerate(sections[1:],1):
        check(idx, section.strip())

    with open('questions.json', 'w', encoding='utf8') as json_file:
        json.dump(jfile, json_file, indent=4, ensure_ascii=False)
    print("Результаты сохранены в файл questions.json")
else:
    print("Не удалось загрузить файл")

Результаты сохранены в файл questions.json


In [3]:
# формирование файла с вопросами
# берем файл questions.json, читаем вопросы, записываем в questions.txt

with open('questions.json', encoding='utf8') as f:
    answers = json.load(f)

with open('questions.txt', 'w', encoding='utf8') as f:
    for questions in answers.items():
        f.write('\n' + questions[1]['name'])
        for question in questions[1]['questions']:
            text = question['вопрос']

            # иногда gpt вопреки инструкциям промпта нумерует вопросы, тогда откусываем начало
            if text[1:3] == '. ':
                text = text[3:]
            if text.startswith('Вопрос '):
                text = text[10:]
            f.write('\n    ' + text)
    print('Создан файл с вопросами questions.txt')

Создан файл с вопросами questions.txt
